In [1]:
import os

In [2]:
import logging

logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

log = logging.getLogger(__name__)

In [3]:
import sys


def in_kaggle():
    return "kaggle_web_client" in sys.modules


log.info(f"in_kaggle: {in_kaggle()}")

2023-03-23 10:01:54,570 [INFO] [365646948] in_kaggle: False


In [4]:
##### INSERT SOURCE CODE HERE FOR SUBMISSION #####

# PLACE HOLDER

In [5]:
if in_kaggle():
    !cp -r /kaggle/input/graphnet-and-dependencies/software .
    !pip install -q /kaggle/working/software/dependencies/torch-1.11.0+cu115-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_cluster-1.6.0-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_sparse-0.6.13-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_geometric-2.2.0.tar.gz
    !cd software/graphnet; pip install -q --no-index --find-links="/kaggle/working/software/dependencies" -e .[torch]

    !cp -r /kaggle/input/hydra-core /kaggle/working
    !mv /kaggle/working/hydra-core/antlr4-python3-runtime-4.9.3.tar.gz.tmp /kaggle/working/hydra-core/antlr4-python3-runtime-4.9.3.tar.gz
    !pip install -q /kaggle/working/hydra-core/*

else:
    %cd ..

/home/jovyan/working


In [6]:
!python 11-inference-parquet.py settings.in_kaggle={in_kaggle()}

graphnet: INFO     2023-03-23 10:01:57 - get_logger - Writing log to logs/graphnet_20230323-100157.log
/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE
  warn(f"Failed to load image Python extension: {e}")
2023-03-23 10:01:58.225361: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-23 10:01:58.401083: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the

In [7]:
!python 15-validation.py settings.in_kaggle={in_kaggle()}

2023-03-23 10:09:46,601 [INFO][utils] Output directory: output/2023-03-23_10-09-46
2023-03-23 10:09:46,602 [INFO][utils] Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
2023-03-23 10:09:46,602 [INFO][utils] PyTorch version: 1.11.0+cu115
2023-03-23 10:09:46,602 [INFO][utils] Fix seed: 440
2023-03-23 10:09:46,980 [INFO][15-validation] Base score: 1.0182490807456754
2023-03-23 10:09:46,991 [INFO][15-validation] Num of low sigma events: 51593, Num of high sigma events: 148407
2023-03-23 10:09:47,039 [INFO][15-validation] Low sigma score: 0.20877596823773575, High sigma score: 1.2996586254007265
2023-03-23 10:09:47,302 [INFO][15-validation] Done.


In [8]:
!python 11-inference-parquet.py settings.in_kaggle={in_kaggle()} training_params.stage2=True

graphnet: INFO     2023-03-23 10:09:50 - get_logger - Writing log to logs/graphnet_20230323-100950.log
/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE
  warn(f"Failed to load image Python extension: {e}")
2023-03-23 10:09:51.446962: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-23 10:09:51.641485: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the

In [9]:
!python 15-validation.py settings.in_kaggle={in_kaggle()} training_params.stage2=True

2023-03-23 10:12:45,386 [INFO][utils] Output directory: output/2023-03-23_10-12-45
2023-03-23 10:12:45,386 [INFO][utils] Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
2023-03-23 10:12:45,386 [INFO][utils] PyTorch version: 1.11.0+cu115
2023-03-23 10:12:45,386 [INFO][utils] Fix seed: 440
2023-03-23 10:12:45,490 [INFO][15-validation] Base score: 0.20902303516008708
2023-03-23 10:12:45,493 [INFO][15-validation] Num of low sigma events: 51251, Num of high sigma events: 342
2023-03-23 10:12:45,511 [INFO][15-validation] Low sigma score: 0.20501445372633975, High sigma score: 0.8097359213033148
2023-03-23 10:12:45,511 [INFO][15-validation] Done.


In [17]:
# !python 12-inference-linear-regression.py settings.in_kaggle={in_kaggle()}

2023-03-23 12:29:49,367 [INFO][utils] Output directory: output/2023-03-23_12-29-49
2023-03-23 12:29:49,367 [INFO][utils] Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
2023-03-23 12:29:49,367 [INFO][utils] PyTorch version: 1.11.0+cu115
2023-03-23 12:29:49,367 [INFO][utils] Fix seed: 440
2023-03-23 12:29:55,460 [INFO][12-inference-linear-regression] Batch [391] ...
100%|██████████████████████████████████| 148407/148407 [04:39<00:00, 530.20it/s]
2023-03-23 12:34:40,825 [INFO][12-inference-linear-regression] Make submission.
2023-03-23 12:34:42,850 [INFO][12-inference-linear-regression] results columns: Index(['event_id', 'azimuth_x', 'zenith_x', 'direction_x', 'direction_y',
       'direction_z', 'azimuth_y', 'zenith_y'],
      dtype='object')
2023-03-23 12:34:42,850 [INFO][12-inference-linear-regression] submission columns: Index(['azimuth', 'zenith'], dtype='object')
2023-03-23 12:34:42,850 [INFO][12-inference-linear-regression] Done.


In [18]:
# !python 15-validation.py settings.in_kaggle={in_kaggle()} training_params.linear_regression=True

2023-03-23 12:36:34,746 [INFO][utils] Output directory: output/2023-03-23_12-36-34
2023-03-23 12:36:34,746 [INFO][utils] Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
2023-03-23 12:36:34,746 [INFO][utils] PyTorch version: 1.11.0+cu115
2023-03-23 12:36:34,746 [INFO][utils] Fix seed: 440
2023-03-23 12:36:34,747 [INFO][15-validation] Validation linear regression.
2023-03-23 12:36:35,034 [INFO][15-validation] Base score: 1.5496008113343298
2023-03-23 12:36:35,034 [INFO][15-validation] Done.


In [10]:
!python 20-ensemble.py settings.in_kaggle={in_kaggle()}

2023-03-23 10:12:48,731 [INFO][utils] Output directory: output/2023-03-23_10-12-48
2023-03-23 10:12:48,731 [INFO][utils] Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
2023-03-23 10:12:48,731 [INFO][utils] PyTorch version: 1.11.0+cu115
2023-03-23 10:12:48,731 [INFO][utils] Fix seed: 440
2023-03-23 10:12:49,237 [INFO][20-ensemble] Base score: 1.0178417468823535
2023-03-23 10:12:49,264 [INFO][20-ensemble] Num of low sigma events: 51401, Num of high sigma events: 148599
2023-03-23 10:12:49,312 [INFO][20-ensemble] Low sigma score: 0.2080248179986346, High sigma score: 1.2990279614745834
2023-03-23 10:12:49,312 [INFO][20-ensemble] Done.
